In [21]:
import loaders.arxiv as arxiv
# import Client, Search
client = arxiv.Client(page_size=2000, )

In [22]:
from urllib.parse import urlencode, quote

query = 'influence functions'
# url = "https://export.arxiv.org/api/query?{}"
# url_params = {
#     'search_query' : q
# }

# urlparams = urlencode(url_params, quote_via=quote)
# url.format(urlparams)
# urllib.parse.(q)

In [23]:
from typing import List
## restricting my search to category computer science
# query = 'cat:Computer Science(cs) AND ti:influence functions'
search = arxiv.Search(query=query, max_results=100, sort_by=arxiv.SortCriterion.SubmittedDate)
results = client.results(search=search)

In [24]:
res = [r for r in results]

In [25]:
for r in res[-5:]:
    print(r.title, r.primary_category, r.published)

The transcritical Bogdanov Takens bifurcation with boundary due to the risk perception on a recruitment epidemiological model math.DS 2024-03-21 05:42:42+00:00
Evolving Benchmark Functions to Compare Evolutionary Algorithms via Genetic Programming cs.NE 2024-03-21 05:42:17+00:00
Genetic Programming for Explainable Manifold Learning cs.NE 2024-03-21 05:17:22+00:00
Temperature dependent conductivity, dielectric relaxation, electrical modulus and impedance spectroscopy of Ni substituted Na$_{3+2x}$Zr$_{2-x}$Ni$_{x}$Si$_2$PO$_{\rm 12}$ cond-mat.mtrl-sci 2024-03-21 05:12:02+00:00
3D Object Detection from Point Cloud via Voting Step Diffusion cs.CV 2024-03-21 05:04:52+00:00


In [286]:
import regex as re
metadf = pd.read_csv('../pdfs/all_arxiv_metadata.csv')
# metadf['arXiv ID'] = metadf['arXiv ID'].str.replace(r'v\d+$', '', regex=True)

refdf = pd.read_csv('../pdfs/paper_references.csv')
refdf['paper'] = refdf['paper'].str.replace(r'^arxiv_pdfs/', '', regex=True).str.replace(r'v\d+\.pdf$', '', regex=True)
refdf['refers'] = refdf['refers'].apply(lambda x: ",".join([re.sub(r'^\s*arXiv:', '', e) for e in x.split(',')]))

conns = set()
papers = set()
res = set()
for index, row in refdf.iterrows():
    for p in row['refers'].split(','):
        if row['paper'] != p:
            papers.add(row['paper'])
            papers.add(p)
            res.add((row['paper'], p))

In [203]:
metadf.drop('Unnamed: 0', axis=1, inplace=True)

In [204]:
metadf.head()

,arXiv ID,Authors,Published,Abstract
0,2403.07865,"Qibing Ren, Chang Gao, Jing Shao, Junchi Yan, ...",2024-03-12T17:55:38Z,The rapid advancement of Large Language Models...
1,2403.07872,"Fangyun Wei, Xi Chen, Lin Luo",2024-03-12T17:59:48Z,"Despite their sophisticated capabilities, larg..."
2,2403.07874,"Lei Zhu, Fangyun Wei, Yanye Lu",2024-03-12T17:59:51Z,"In this work, we investigate the potential of ..."
3,2403.07974,"Naman Jain, King Han, Alex Gu, Wen-Ding Li, Fa...",2024-03-12T17:58:04Z,Large Language Models (LLMs) applied to code-r...
4,2403.08004,"Rodrigo Santos, João Silva, António Branco",2024-03-12T18:12:50Z,The combination of language processing and ima...


In [205]:
refdf.head()

,paper,refers
0,2403.07865,2403.07865
1,2403.07872,"2403.07872,2303.08774,2311.16867,2108.07732,16..."
2,2403.07874,"2403.07874,2305.10403,1504.00325,2204.02311,18..."
3,2403.07974,"2403.07974,1910.02216,2301.03988,1808.01400,22..."
4,2403.08004,"2403.08004,1409.0473,2005.14165,2010.11929,210..."


In [282]:
from neo4j import GraphDatabase

uri = "neo4j+s://c6666d76.databases.neo4j.io:7687"  # Replace with your Neo4j database URI
username = "neo4j"  # Replace with your Neo4j username

driver = GraphDatabase.driver(uri, auth=(username, password))

In [211]:
def create_node(tx, row):
    query = """
    MERGE (p:Paper {arxiv_id: $arxiv_id})
    ON CREATE SET p.authors = $authors, p.published = $published, p.abstract = $abstract
    RETURN p
    """
    result = tx.run(query, arxiv_id=row['arXiv ID'], authors=row['Authors'], published=row['Published'], abstract=row['Abstract'])
    return result.single()['p']

In [221]:
def create_author_paper_connections(tx, row):
    arxiv_id = row['arXiv ID']
    authors = [author.strip() for author in row['Authors'].split(',')]
    
    for author in authors:
        query = """
        MATCH (p:Paper {arxiv_id: $arxiv_id})
        MERGE (a:Author {name: $author})
        MERGE (a)-[:AUTHORED]->(p)
        """
        tx.run(query, arxiv_id=arxiv_id, author=author)

In [280]:
# def create_paper_references(tx, paper_refs):
#     query = """
#     MATCH (p1:Paper {arxiv_id: $paper_id})
#     MATCH (p2:Paper {arxiv_id: $ref_id})
#     MERGE (p1)-[:REFERENCES]->(p2)
#     """
#     for paper_ref in paper_refs:
#         paper_id = paper_ref[0]
#         ref_id = paper_ref[1]
#         tx.run(query, paper_id=paper_id, ref_id=ref_id)

def create_paper_connections(tx, pair_list):
    for pair in pair_list:
        arxiv_id1, arxiv_id2 = pair
        arxiv_id1 = float(arxiv_id1)
        arxiv_id2 = float(arxiv_id2)
        query = """
        MATCH (p1:Paper {arxiv_id: $arxiv_id1})
        MATCH (p2:Paper {arxiv_id: $arxiv_id2})
        MERGE (p2)-[:CITES]->(p1)
        """
        tx.run(query, arxiv_id1=arxiv_id1, arxiv_id2=arxiv_id2)



In [212]:
with driver.session() as session:
    for index, row in metadf.iterrows():
        session.execute_write(create_node, row)

In [287]:
res = list(res)

In [288]:
with driver.session() as session:
    try:
        session.write_transaction(create_paper_connections, res)
        print("Connections created successfully.")
    except Exception as e:
        print(f"Error creating connections: {str(e)}")


/var/folders/4q/1ndm9jyx44d9v50kxp2r2mnr0000gn/T/ipykernel_21272/1733800859.py:3: DeprecationWarning: write_transaction has been renamed to execute_write
  session.write_transaction(create_paper_connections, res)


Connections created successfully.


In [222]:
with driver.session() as session:
    for index, row in metadf.iterrows():
        session.execute_write(create_author_paper_connections, row)

In [187]:
driver.close()

In [225]:
p1 = '2403.12403'
p2= '2403.08035'
float(p1), float(p2)

(2403.12403, 2403.08035)

In [ ]:
for i in metadf.index:
    

In [224]:
with driver.session() as session:
    session.write_transaction(create_paper_connections, [(p1, p2)])


/var/folders/4q/1ndm9jyx44d9v50kxp2r2mnr0000gn/T/ipykernel_21272/460415058.py:2: DeprecationWarning: write_transaction has been renamed to execute_write
  session.write_transaction(create_paper_connections, [(p1, p2)])


In [140]:
# papers = set()
# for p1, p2 in res:
#     papers.add(p1)
#     papers.add(p2)
# papers = list(papers)

In [189]:
metadf.set_index('arXiv ID', inplace=True)

In [215]:
for r in list(res):
    print(r)

('2403.10779', '2401.00125')
('2403.13447', '2302.14045')
('2403.14409', '1606.06031')
('2403.09572', '2307.14539')
('2403.13027', '1910.03771')
('2403.13583', '2301.03988')
('2403.09606', '2303.02536')
('2403.12936', '2303.08774')
('2403.12446', '2308.12950')
('2403.09527', '2303.17395')
('2403.14151', '1907.07062')
('2403.08281', '2306.02561')
('2403.11057', '2305.10601')
('2403.08282', '2305.16960')
('2403.14238', '2308.03188')
('2403.12403', '2107.07229')
('2403.14151', '2008.10349')
('2403.09972', '2306.13063')
('2403.12910', '2309.11564')
('2403.11807', '2305.16867')
('2403.13485', '2311.08721')
('2403.13257', '2110.06296')
('2403.10553', '2306.17439')
('2403.11128', '2302.04761')
('2403.11330', '2212.09746')
('2403.13309', '2401.05778')
('2403.14151', '1908.00024')
('2403.13309', '2305.14710')
('2403.09751', '2007.05285')
('2403.12958', '2311.06062')
('2403.12196', '2301.08653')
('2403.12801', '2305.03726')
('2403.11124', '2308.05696')
('2403.11401', '2302.07241')
('2403.08730',

In [143]:
from neo4j import GraphDatabase

uri = "neo4j+s://c6666d76.databases.neo4j.io:7687"  # Replace with your Neo4j database URI
username = "neo4j"  # Replace with your Neo4j username
password = "QtieY50I9jrC5XNnDPq6c-YryA0cEz-qC3LjoKb7-9A"  # Replace with your Neo4j password

driver = GraphDatabase.driver(uri, auth=(username, password))

def find_missing_files(tx, arxiv_ids):
    query = """
    MATCH (p:Paper)
    WHERE p.arxiv_id IN $arxiv_ids
    RETURN p.arxiv_id AS existing_arxiv_id
    """
    result = tx.run(query, arxiv_ids=arxiv_ids)
    existing_arxiv_ids = [record["existing_arxiv_id"] for record in result]
    missing_arxiv_ids = set(arxiv_ids) - set(existing_arxiv_ids)
    return list(missing_arxiv_ids)

# List of papers
# papers = [
#     "2403.07872",
#     "2303.08774",
#     "2311.16867",
#     "2305.08387",
#     "2305.08388",
#     "2305.08389"
# ]

with driver.session() as session:
    missing_files = session.read_transaction(find_missing_files, papers)

# print("Missing files:")
# for file in missing_files:
#     print(file)

driver.close()

/var/folders/4q/1ndm9jyx44d9v50kxp2r2mnr0000gn/T/ipykernel_21272/3797796689.py:31: DeprecationWarning: read_transaction has been renamed to execute_read
  missing_files = session.read_transaction(find_missing_files, papers)


In [146]:
pd.DataFrame(missing_files).to_csv('../pdfs/missing.csv')

In [147]:
# import arxiv

# def fetch_metadata(arxiv_ids):
#     metadata = []
    
#     for arxiv_id in arxiv_ids:
#         try:
#             search = arxiv.Search(id_list=[arxiv_id])
#             paper = next(search.results())
            
#             metadata.append({
#                 "arxiv_id": arxiv_id,
#                 "title": paper.title,
#                 "authors": [author.name for author in paper.authors],
#                 "summary": paper.summary,
#                 "published": paper.published.strftime("%Y-%m-%d"),
#                 "doi": paper.doi,
#                 "primary_category": paper.primary_category,
#                 "categories": paper.categories
#             })
#         except StopIteration:
#             print(f"Paper not found: {arxiv_id}")
#         except Exception as e:
#             print(f"Error fetching metadata for {arxiv_id}: {str(e)}")
    
#     return metadata

# # List of arXiv paper IDs
# arxiv_ids = [
#     "2403.07872",
#     "2303.08774",
#     "2311.16867",
#     "2305.08387",
#     "2305.08388",
#     "2305.08389"
# ]

# metadata = fetch_metadata(missing_files)

# for paper in metadata[:5]:
#     print("arXiv ID:", paper["arxiv_id"])
#     print("Title:", paper["title"])
#     print("Authors:", ", ".join(paper["authors"]))
#     print("Summary:", paper["summary"])
#     print("Published:", paper["published"])
#     print("DOI:", paper["doi"])
#     print("Primary Category:", paper["primary_category"])
#     print("Categories:", ", ".join(paper["categories"]))
#     print("---")

https://export.arxiv.org/api/query?search_query=&id_list=2304.11082&sortBy=relevance&sortOrder=descending&start=0&max_results=100
https://export.arxiv.org/api/query?search_query=&id_list=2309.17288&sortBy=relevance&sortOrder=descending&start=0&max_results=100
https://export.arxiv.org/api/query?search_query=&id_list=2306.15195&sortBy=relevance&sortOrder=descending&start=0&max_results=100
https://export.arxiv.org/api/query?search_query=&id_list=2308.01552&sortBy=relevance&sortOrder=descending&start=0&max_results=100
https://export.arxiv.org/api/query?search_query=&id_list=2310.05719&sortBy=relevance&sortOrder=descending&start=0&max_results=100
https://export.arxiv.org/api/query?search_query=&id_list=2312.09545&sortBy=relevance&sortOrder=descending&start=0&max_results=100
https://export.arxiv.org/api/query?search_query=&id_list=2306.13298&sortBy=relevance&sortOrder=descending&start=0&max_results=100
https://export.arxiv.org/api/query?search_query=&id_list=2309.09971&sortBy=relevance&sortO

In [157]:
add_df = pd.DataFrame(metadata)

In [158]:
# arXiv ID,Authors,Published,Abstract

add_df.rename(columns={'arxiv_id': 'arXiv ID', 'authors': 'Authors', 'published': 'Published', 'summary': 'Abstract'}, inplace=True)

In [159]:
add_df.head()

,arXiv ID,title,Authors,Abstract,Published,doi,primary_category,categories
0,2306.10675,LaDe: The First Comprehensive Last-mile Delive...,"[Lixia Wu, Haomin Wen, Haoyuan Hu, Xiaowei Mao...",Real-world last-mile delivery datasets are cru...,2023-06-19,None,cs.DB,"[cs.DB, cs.AI]"
1,2301.00910,Causal Inference in Recommender Systems: A Sur...,"[Yaochen Zhu, Jing Ma, Jundong Li]","In the era of information overload, recommende...",2023-01-03,None,cs.IR,[cs.IR]
2,2306.08997,Exploring the MIT Mathematics and EECS Curricu...,"[Sarah J. Zhang, Samuel Florin, Ariel N. Lee, ...","We curate a comprehensive dataset of 4,550 que...",2023-06-15,None,cs.CL,"[cs.CL, cs.AI, cs.LG]"
3,2310.00785,BooookScore: A systematic exploration of book-...,"[Yapei Chang, Kyle Lo, Tanya Goyal, Mohit Iyyer]",Summarizing book-length documents (>100K token...,2023-10-01,None,cs.CL,"[cs.CL, cs.AI, cs.LG]"
4,2401.08699,On Image Search in Histopathology,"[H. R. Tizhoosh, Liron Pantanowitz]",Pathology images of histopathology can be acqu...,2024-01-14,None,eess.IV,"[eess.IV, cs.AI, cs.CV, cs.IR, q-bio.QM]"


In [160]:
# arXiv ID,Authors,Published,Abstract
add_df.drop(columns=['doi', 'categories', 'primary_category', 'title'], inplace=True)
add_df.head()

,arXiv ID,Authors,Abstract,Published
0,2306.10675,"[Lixia Wu, Haomin Wen, Haoyuan Hu, Xiaowei Mao...",Real-world last-mile delivery datasets are cru...,2023-06-19
1,2301.00910,"[Yaochen Zhu, Jing Ma, Jundong Li]","In the era of information overload, recommende...",2023-01-03
2,2306.08997,"[Sarah J. Zhang, Samuel Florin, Ariel N. Lee, ...","We curate a comprehensive dataset of 4,550 que...",2023-06-15
3,2310.00785,"[Yapei Chang, Kyle Lo, Tanya Goyal, Mohit Iyyer]",Summarizing book-length documents (>100K token...,2023-10-01
4,2401.08699,"[H. R. Tizhoosh, Liron Pantanowitz]",Pathology images of histopathology can be acqu...,2024-01-14


In [161]:
df = pd.read_csv('../pdfs/arxiv_metadata.csv')
df['arXiv ID'] = df['arXiv ID'].str.replace(r'v\d+$', '', regex=True)
df.head()

,arXiv ID,Authors,Published,Abstract
0,2403.07865,"Qibing Ren, Chang Gao, Jing Shao, Junchi Yan, ...",2024-03-12T17:55:38Z,The rapid advancement of Large Language Models...
1,2403.07872,"Fangyun Wei, Xi Chen, Lin Luo",2024-03-12T17:59:48Z,"Despite their sophisticated capabilities, larg..."
2,2403.07874,"Lei Zhu, Fangyun Wei, Yanye Lu",2024-03-12T17:59:51Z,"In this work, we investigate the potential of ..."
3,2403.07974,"Naman Jain, King Han, Alex Gu, Wen-Ding Li, Fa...",2024-03-12T17:58:04Z,Large Language Models (LLMs) applied to code-r...
4,2403.08004,"Rodrigo Santos, João Silva, António Branco",2024-03-12T18:12:50Z,The combination of language processing and ima...


In [165]:
all_df = pd.concat([df, add_df], axis=0)

In [166]:
all_df.to_csv('../pdfs/all_arxiv_metadata.csv')

In [230]:
metadf.Abstract

0       The rapid advancement of Large Language Models...
1       Despite their sophisticated capabilities, larg...
2       In this work, we investigate the potential of ...
3       Large Language Models (LLMs) applied to code-r...
4       The combination of language processing and ima...
                              ...                        
2995    In recent years, autonomous agents have surged...
2996    Recently, large language models such as ChatGP...
2997    In this study, we assess the efficacy of emplo...
2998    Large language models (LLMs) are trained to im...
2999    Large language models (LLMs) such as ChatGPT a...
Name: Abstract, Length: 3000, dtype: object

In [318]:
from langchain.chains import GraphCypherQAChain
from langchain_community.graphs import Neo4jGraph
from langchain_openai import ChatOpenAI
from langchain_anthropic import ChatAnthropic

graph = Neo4jGraph(
    url=uri, username=username, password=password
)
graph.schema

'Node properties are the following:\nPaper {abstract: STRING, arxiv_id: FLOAT, authors: STRING, published: STRING},Author {name: STRING}\nRelationship properties are the following:\n\nThe relationships are the following:\n(:Paper)-[:CITES]->(:Paper),(:Author)-[:AUTHORED]->(:Paper)'

In [325]:
# chain = GraphCypherQAChain.from_llm(
#     ChatAnthropic(temperature=0, model_name='claude-3-haiku-20240307'), graph=graph, verbose=True, top_k=3, return_intermediate_steps=True
# )
chain = GraphCypherQAChain.from_llm(
    ChatOpenAI(temperature=0), graph=graph, verbose=True, top_k=3, return_intermediate_steps=False, return_direct=True
)
query = 'papers on large language models'

res = chain(query)



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (p:Paper)-[:CITES]->(:Paper)
WHERE p.abstract CONTAINS 'large language models'
RETURN p

> Finished chain.


In [330]:
[['https://arxiv.org/abs' + str(r['p']['arxiv_id']), r['p']['abstract']] for r in res['result']]

[['https://arxiv.org/abs2306.08997',
  "We curate a comprehensive dataset of 4,550 questions and solutions from\nproblem sets, midterm exams, and final exams across all MIT Mathematics and\nElectrical Engineering and Computer Science (EECS) courses required for\nobtaining a degree. We evaluate the ability of large language models to fulfill\nthe graduation requirements for any MIT major in Mathematics and EECS. Our\nresults demonstrate that GPT-3.5 successfully solves a third of the entire MIT\ncurriculum, while GPT-4, with prompt engineering, achieves a perfect solve rate\non a test set excluding questions based on images. We fine-tune an open-source\nlarge language model on this dataset. We employ GPT-4 to automatically grade\nmodel responses, providing a detailed performance breakdown by course,\nquestion, and answer type. By embedding questions in a low-dimensional space,\nwe explore the relationships between questions, topics, and classes and\ndiscover which questions and classes 

In [279]:
res['result']

'The authors of the paper with arxiv_id 2403.09606 are Xiaoyu Liu and Paiheng Xu. According to the provided information, Xiaoyu Liu has also written another paper with arxiv_id 2403.08743.'

In [333]:
import arxiv
from langchain.llms import OpenAI
from langchain.chains import RetrievalQA
from langchain.document_loaders import ArxivLoader
from langchain.vectorstores import Chroma
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from dotenv import load_dotenv
import os

load_dotenv()
# Set up OpenAI API key
openai_api_key = os.environ['OPENAI_API_KEY']
from langchain_community.retrievers import ArxivRetriever
query = 'large language models are really useful when it comes to helping users summarize stuff'
retriever = ArxivRetriever(load_max_docs=5)
docs = retriever.get_relevant_documents(query=query)
print(len(docs))
print([[doc.metadata['Entry ID'], doc.metadata['Title']] for doc in docs])

https://export.arxiv.org/api/query?search_query=large+language+models+are+really+useful+when+it+comes+to+helping+users+summarize+stuff&id_list=&sortBy=relevance&sortOrder=descending&start=0&max_results=100
https://export.arxiv.org/api/query?search_query=large+language+models+are+really+useful+when+it+comes+to+helping+users+summarize+stuff&id_list=&sortBy=relevance&sortOrder=descending&start=0&max_results=100
https://export.arxiv.org/api/query?search_query=large+language+models+are+really+useful+when+it+comes+to+helping+users+summarize+stuff&id_list=&sortBy=relevance&sortOrder=descending&start=0&max_results=100
3
[['http://arxiv.org/abs/2306.05537v1', 'AaKOS: Aspect-adaptive Knowledge-based Opinion Summarization'], ['http://arxiv.org/abs/2304.06815v3', 'Automatic Semantic Augmentation of Language Model Prompts (for Code Summarization)'], ['http://arxiv.org/abs/2104.12465v1', 'GPT2MVS: Generative Pre-trained Transformer-2 for Multi-modal Video Summarization']]


In [315]:
from langchain.chains import ConversationalRetrievalChain
from langchain_openai import ChatOpenAI

model = ChatOpenAI(model_name="gpt-3.5-turbo")  # switch to 'gpt-4'
qa = RetrievalQA.from_llm(model, retriever=retriever)
result = qa(query)


https://export.arxiv.org/api/query?search_query=large+language+models+are+really+useful+when+it+comes+to+helping+users+summarize+stuff&id_list=&sortBy=relevance&sortOrder=descending&start=0&max_results=100
https://export.arxiv.org/api/query?search_query=large+language+models+are+really+useful+when+it+comes+to+helping+users+summarize+stuff&id_list=&sortBy=relevance&sortOrder=descending&start=0&max_results=100
https://export.arxiv.org/api/query?search_query=large+language+models+are+really+useful+when+it+comes+to+helping+users+summarize+stuff&id_list=&sortBy=relevance&sortOrder=descending&start=0&max_results=100


In [316]:
result

{'query': 'large language models are really useful when it comes to helping users summarize stuff',
 'result': 'Large Language Models (LLMs) like ChatGPT have shown potential in text generation tasks like text summarization. They can help users quickly retrieve relevant information by generating concise content from long documents. However, implementing LLMs for offline applications can be challenging due to the massive data and resources they require. Additionally, existing text summarization approaches may lack the adaptive nature needed for capturing diverse aspects in opinion summarization. So, while LLMs are useful, there are still challenges to overcome in their implementation for text summarization tasks.'}